# Inference with EmotionStack

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U adapter-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.1 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html


In [ ]:
from transformers import AutoConfig, AutoModelWithHeads
import torch
from transformers import RobertaTokenizer
import transformers.adapters.composition as ac
import pandas as pd
from tqdm import tqdm
from scipy.stats import pearsonr
import random

In [ ]:
val_data = pd.read_csv('/content/drive/MyDrive/NLP_final_pro/data/messages_dev_features_ready_for_WS_2022.tsv', sep='\t')

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

config = AutoConfig.from_pretrained(
    "roberta-base",
    num_labels=1
)

model = AutoModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassifica

In [ ]:
""" load adapters """
empathy_adapter_path = "/content/drive/MyDrive/NLP_final_pro/trained_adapters/EMP_emotion_stack"
distress_adapter_path = "/content/drive/MyDrive/NLP_final_pro/trained_adapters/DIS_emotion_stack"

empathy_adapter = model.load_adapter(empathy_adapter_path, load_as=empathy_adapter_path.split('/')[-1])
distress_adapter = model.load_adapter(distress_adapter_path, load_as=distress_adapter_path.split('/')[-1])

In [ ]:
model.set_active_adapters(ac.Parallel(empathy_adapter, distress_adapter))
model.active_head = ['EMP_emotion_stack', 'DIS_emotion_stack']

# Inference with Epitome Fusion

In [ ]:
""" load pretrained adapter composition """

from transformers.adapters.composition import Fuse


fusion_path = f"/content/drive/MyDrive/NLP_final_pro/trained_adapters/EpitomeFusion-distress"

# load each individual adapter
dis_er_adapter = model.load_adapter(
                fusion_path + '/distress-emotional-reactions')
dis_ex_adapter = model.load_adapter(
                fusion_path + '/distress-explorations')
dis_ip_adapter = model.load_adapter(
                fusion_path + '/distress-interpretations')

# load adapter fusion
model.load_adapter_fusion(
    fusion_path
)

# set active adapters
model.set_active_adapters(Fuse(dis_er_adapter, dis_ex_adapter, dis_ip_adapter))

# # load head
# path, distress_adapter = model.load_head(fusion_path)

model.active_head = 'EpitomeFusion-distress'

In [ ]:
""" load pretrained adapter composition: empathy """

fusion_path = f"/content/drive/MyDrive/NLP_final_pro/trained_adapters/EpitomeFusion-empathy"

# load each individual adapter
emp_er_adapter = model.load_adapter(
                fusion_path + '/empathy-emotional-reactions'
            )
emp_ex_adapter = model.load_adapter(
                fusion_path + '/empathy-explorations'
            )
emp_ip_adapter = model.load_adapter(
                fusion_path + '/empathy-interpretations'
            )

# load adapter fusion
model.load_adapter_fusion(
    fusion_path
)

# set active adapters
model.set_active_adapters(Fuse(emp_er_adapter, emp_ex_adapter, emp_ip_adapter))

# # load head
# path, empathy_adapter = model.load_head(fusion_path)

In [ ]:
model.active_head = 'EpitomeFusion-empathy'
model.active_head

'EpitomeFusion-empathy'

In [ ]:
def predict_sen(sentence):
    tokens = tokenizer.tokenize(sentence)
    input_ids = torch.tensor(tokenizer.convert_tokens_to_ids(tokens)).to('cuda')

    # move the model to the GPU
    model.to('cuda')

    # set active adapters
    model.set_active_adapters(Fuse(dis_er_adapter, dis_ex_adapter, dis_ip_adapter))
    model.active_head = 'EpitomeFusion-distress'
    outputs = model(input_ids)
    dis_score_f = outputs.logits[0][0].tolist()

    # set active adapters
    model.set_active_adapters(Fuse(emp_er_adapter, emp_ex_adapter, emp_ip_adapter))
    model.active_head = 'EpitomeFusion-empathy'
    outputs = model(input_ids)
    emp_score_f = outputs.logits[0][0].tolist()

    # stack
    model.set_active_adapters(ac.Parallel(empathy_adapter, distress_adapter))
    model.active_head = ['EMP_emotion_stack', 'DIS_emotion_stack']
    outputs = model(input_ids)

    emp_score_s = outputs[0].logits[0][0].tolist()
    dis_score_s = outputs[1].logits[0][0].tolist()

    # move the model back to the CPU
    model.to('cpu')

    return emp_score_f, dis_score_f, emp_score_s, dis_score_s

In [ ]:
def get_pred(val_data):
  pred_emp_f = []
  pred_emp_s = []
  acu_emp = []
  pred_dis_f = []
  pred_dis_s = []
  acu_dis = []
  for idx in tqdm(range(len(val_data))):
      essay = val_data['essay'].values[idx]
      emp_score_f, dis_score_f,emp_score_s,dis_score_s = predict_sen(essay)
    
      pred_emp_f.append(emp_score_f)
      pred_emp_s.append(emp_score_s)
      acu_emp.append(val_data['empathy'].values[idx])
    
      pred_dis_f.append(dis_score_f)
      pred_dis_s.append(dis_score_s)
      acu_dis.append(val_data['distress'].values[idx])
  return pred_emp_f, pred_emp_s, acu_emp, pred_dis_f, pred_dis_s, acu_dis 

In [ ]:
def get_positive_score(l_aug,org,aug):
  wrong_list = []
  for i in range(len(org)):
    if (abs(org[i]-aug[i])>0.2):
      wrong_list.append(i)
  print("score:",len(wrong_list)/len(org))
  random_five = random.sample(wrong_list, 5)
  for j in random_five:
    print(val_data['essay'].values[j])
    print("score:",org[j])
    print(l_aug['essay'].values[j])
    print("score:",aug[j])

In [ ]:
def get_scores(l_aug,pred_emp_f_org, pred_emp_s_org, acu_emp, pred_dis_f_org, pred_dis_s_org, acu_dis,pred_emp_f_aug, pred_emp_s_aug, pred_dis_f_aug, pred_dis_s_aug):
  
  corr_coef_f_aug, p_value = pearsonr(pred_emp_f_aug, acu_emp)
  print("Pearson Fution empathy:", corr_coef_f_aug)

  corr_coef_s_aug, p_value = pearsonr(pred_emp_s_aug, acu_emp)
  print("Pearson Stack empathy:", corr_coef_s_aug)

  corr_coef_f_a, p_value = pearsonr(pred_dis_f_aug, acu_dis)
  print("Pearson Fution distress:", corr_coef_f_a)

  corr_coef_s_a, p_value = pearsonr(pred_dis_s_aug, acu_dis)
  print("Pearson Stack distress:", corr_coef_s_a)

  print("Fution empathy:")
  get_positive_score(l_aug,pred_emp_f_org,pred_emp_f_aug)

  print("Stack empathy:")
  get_positive_score(l_aug,pred_emp_s_org,pred_emp_s_aug)

  print("Fution distress:")
  get_positive_score(l_aug,pred_dis_f_org,pred_dis_f_aug)

  print("Stack distress:")
  get_positive_score(l_aug,pred_dis_s_org,pred_dis_s_aug)

In [ ]:
pred_emp_f_org, pred_emp_s_org, acu_emp, pred_dis_f_org, pred_dis_s_org, acu_dis = get_pred(val_data)

100%|██████████| 270/270 [03:57<00:00,  1.14it/s]


In [ ]:
corr_coef, p_value = pearsonr(pred_emp_s_org, acu_emp)

print("Pearson correlation coefficient emp stack:", corr_coef)

Pearson correlation coefficient emp stack: 0.4890706649247585


In [ ]:
corr_coef, p_value = pearsonr(pred_emp_f_org, acu_emp)

print("Pearson correlation coefficient emp fusion:", corr_coef)

Pearson correlation coefficient emp fusion: 0.5129559021014046


In [ ]:
corr_coef, p_value = pearsonr(pred_dis_s_org, acu_dis)

print("Pearson correlation coefficient dis stack:", corr_coef)

Pearson correlation coefficient dis stack: 0.4537656974443214


In [ ]:
corr_coef, p_value = pearsonr(pred_dis_f_org, acu_dis)

print("Pearson correlation coefficient dis fusion:", corr_coef)

Pearson correlation coefficient dis fusion: 0.4808846695920748


In [ ]:
re_n = pd.read_csv('/content/drive/MyDrive/NLP_final_pro/aug_data/aug_changed_noun.tsv', sep='\t')
pred_emp_f_re_n, pred_emp_s_re_n, acu_emp_re_n, pred_dis_f_re_n, pred_dis_s_re_n, acu_dis_re_n = get_pred(re_n)

100%|██████████| 270/270 [02:51<00:00,  1.57it/s]


In [ ]:
# For changing nouns 
get_scores(re_n,pred_emp_f_org, pred_emp_s_org, acu_emp, pred_dis_f_org, pred_dis_s_org, acu_dis,pred_emp_f_re_n, pred_emp_s_re_n, pred_dis_f_re_n, pred_dis_s_re_n)

Pearson Fution empathy: 0.4485441423190989
Pearson Stack empathy: 0.42293424480423275
Pearson Fution distress: 0.43958276297009913
Pearson Stack distress: 0.3908767086977239
Fution empathy:
score: 0.8222222222222222
Can you believe some people still don't believe in climate change? When will people, especially Americans like Donald Trump, wake up? What is it going to take to get them to start making a difference. We live in Connecticut, imagine our ENTIRE state burning up! What do climate change deniers think is really causing this then? Bunch of idiots in congress who don't care. We are losing our forests and animals and it just so sad.
can you believe some masses still don't believe in clime change? when will people, especially american like donald trump, wake up? what is it going to take to get them to start making a difference. we live in connecticut, imagine our entire province electrocution up! what do climate alteration denier think is really causing this then? clump of cretin i

In [ ]:
re_v = pd.read_csv('/content/drive/MyDrive/NLP_final_pro/aug_data/aug_changed_noun.tsv', sep='\t')
pred_emp_f_re_v, pred_emp_s_re_v, acu_emp_re_v, pred_dis_f_re_v, pred_dis_s_re_v, acu_dis_re_v = get_pred(re_v)

100%|██████████| 270/270 [02:57<00:00,  1.52it/s]


In [ ]:
# For changing verds
get_scores(re_v,pred_emp_f_org, pred_emp_s_org, acu_emp, pred_dis_f_org, pred_dis_s_org, acu_dis,pred_emp_f_re_v, pred_emp_s_re_v, pred_dis_f_re_v, pred_dis_s_re_v)

Pearson Fution empathy: 0.4485441423190989
Pearson Stack empathy: 0.42293424480423275
Pearson Fution distress: 0.43958276297009913
Pearson Stack distress: 0.3908767086977239
Fution empathy:
score: 0.8222222222222222
Did you see the article about the airline worker who was shot at work. Unbelievable that these things keep happening. The guy who did the shooting is also dead killed himself. I feel so bad for the family of the man who was killed. I guess his son plays for the Chiefs. I hope they will figure out why it happened what this guy had against the man he shot. Terrible and I thought airports had better security than to let a man with a gun get there and be able to kill someone. It's so crazy nowadays that you aren't safe anywhere. I hope this won't start more aint gun talk. We both know its not the gun but the person doing the shooting. I wish we could concentrate more on mental health and not so much omg it was the gun. The gun was just the instrument not the reason the man is d